In [24]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score,recall_score, f1_score
import re
import string 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
from nltk.tokenize import word_tokenize



In [25]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/refs/heads/main/tweet_emotions.csv").drop(columns=['tweet_id'])
df.head()

sentiment                                            content
0       empty  @tiffanylue i know  i was listenin to bad habi...
1     sadness  Layin n bed with a headache  ughhhh...waitin o...
2     sadness                Funeral ceremony...gloomy friday...
3  enthusiasm               wants to hang out with friends SOON!
4     neutral  @dannycastillo We want to trade with someone w...

In [26]:
def clean_data(text):
  text = text.lower()
  text = text.translate(str.maketrans('','',string.punctuation))
  text = re.sub(r'\d+', '', text)
  text = text.strip()
  return text

def noise_removal(text):
  text = re.sub(r"http\S+|www\S+|https\S+",'',text)
  text = re.sub(r'[^A-Za-z\s]','',text)
  return text

def tokenization(text):
  tokens = word_tokenize(text)
  return tokens

def stopword_removal(tokens):
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  return tokens

def lemmatization(tokens):
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens


def preprocess_text(df):
  df['content'] = df['content'].apply(lambda content: clean_data(content))
  df['content'] = df['content'].apply(lambda content: noise_removal(content))
  df['content'] = df['content'].apply(lambda content:tokenization(content))
  df['content'] = df['content'].apply(lambda content: stopword_removal(content))
  df['content'] = df['content'].apply(lambda content: lemmatization(content))
  df['content'] = df['content'].apply(lambda tokens: ' '.join(tokens))
  return df 
  

In [27]:
df = preprocess_text(df)
df.head()

sentiment                                            content
0       empty  tiffanylue know listenin bad habit earlier sta...
1     sadness             layin n bed headache ughhhhwaitin call
2     sadness                      funeral ceremonygloomy friday
3  enthusiasm                              want hang friend soon
4     neutral  dannycastillo want trade someone houston ticke...

In [28]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [29]:
x= df['sentiment'].isin(['happiness','sadness'])
df = df[x]

In [30]:
df['sentiment'] = df['sentiment'].replace({'sadness':0,'happiness':1})
df.head()

C:\Users\hp\AppData\Local\Temp\ipykernel_22348\1083041448.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0,'happiness':1})


sentiment                                            content
1          0             layin n bed headache ughhhhwaitin call
2          0                      funeral ceremonygloomy friday
6          0  sleep im thinking old friend want he married d...
8          0                       charviray charlene love miss
9          0                     kelcouch im sorry least friday

In [31]:
vectorizer = CountVectorizer(max_features=1000)
x = vectorizer.fit_transform(df['content'])
y = df['sentiment']


In [32]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state = 42)

In [33]:
import dagshub

dagshub.init(repo_owner='Nite2005', repo_name='mlops-mini-project', mlflow=True)
mlflow.set_tracking_uri("https://dagshub.com/Nite2005/mlops-mini-project.mlflow")


mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "Nite2005/mlops-mini-project"

Repository Nite2005/mlops-mini-project initialized!

<Experiment: artifact_location='mlflow-artifacts:/decd79af8b5f4f27a447874f440c4d82', creation_time=1752939822809, experiment_id='0', last_update_time=1752939822809, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [38]:
with mlflow.start_run():

    mlflow.log_param("vectorizer","Bag of words")
    mlflow.log_param("num_features",1000)
    mlflow.log_param("test_size",0.2)


    model = LogisticRegression()

    model.fit(x_train,y_train)

    mlflow.log_param('model',"Logistic Regression")

    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    recall = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)

    mlflow.log_metric('accuracy',accuracy)
    mlflow.log_metric('precission',precision)
    mlflow.log_metric('recall',recall)
    mlflow.log_metric('f1_score',f1)


    # mlflow.sklearn.log_model(model,'model')
    import joblib
    joblib.dump(model, "model.pkl")  # Save model as pickle

    mlflow.log_artifact("model.pkl")  # Upload as artifact

    import os
    notebook_path = 'exp1_baseline_model.ipynb'
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    
    mlflow.log_artifact(notebook_path)
    print(f"Accuracy:{accuracy}")
    print(f"precission: {precision}")
    print(f"Recall: {recall}")
    print(f"f1_score: {f1}")

Accuracy:0.7826506024096386
precission: 0.7635514018691589
Recall: 0.8049261083743843
f1_score: 0.7836930455635491
🏃 View run kindly-toad-739 at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/0/runs/1cc30f14e3a748b8b771ed1156392e0b
🧪 View experiment at: https://dagshub.com/Nite2005/mlops-mini-project.mlflow/#/experiments/0
